In [ ]:
!pip uninstall pandas --y
!pip install pandas==1.3.0

Found existing installation: pandas 1.1.5
Uninstalling pandas-1.1.5:
  Successfully uninstalled pandas-1.1.5
     |████████████████████████████████| 10.8 MB 34.1 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas~=1.1.0; python_version >= "3.0", but you have pandas 1.3.0 which is incompatible.


In [2]:
import numpy as np
import pandas as pd
import pickle


# Acquire 500 coins (layer 1) and 500 tokens (layer 2) from Coinmarketcap

In [ ]:
from requests import Request, Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
import json

url = 'https://pro-api.coinmarketcap.com/v1/cryptocurrency/listings/latest'
coin_parameters = {
  'start':'1',
  'limit':'500',
  'convert':'USD',
  'cryptocurrency_type':'coins'
}
token_parameters = {
  'start':'1',
  'limit':'500',
  'convert':'USD',
  'cryptocurrency_type':'tokens'
}
headers = {
  'Accepts': 'application/json',
  'X-CMC_PRO_API_KEY': 'c3e1acb4-22ad-49f1-8819-515ce1702ab9',
}

In [ ]:
session = Session()
session.headers.update(headers)

try:
  coin_response = session.get(url, params=coin_parameters)
  coin_data = json.loads(coin_response.text)
  token_response = session.get(url, params=token_parameters)
  token_data = json.loads(token_response.text)
  print('Data Fetched Successfully!')
except (ConnectionError, Timeout, TooManyRedirects) as e:
  print(e)

Data Fetched Successfully!


In [ ]:
# save data
# with open('500_coins.pickle', 'wb') as outfile:
#     pickle.dump(coin_data, outfile, protocol=pickle.HIGHEST_PROTOCOL)
# with open('500_tokens.pickle', 'wb') as outfile:
#     pickle.dump(token_data, outfile, protocol=pickle.HIGHEST_PROTOCOL)

# read saved data
with open('/content/500_coins.pickle', 'rb') as handle1:
    coin_data = pickle.load(handle1)
with open('/content/500_tokens.pickle', 'rb') as handle2:
    token_data = pickle.load(handle2)

Data as of 2021.12.11

In [ ]:
full_df = pd.DataFrame()

type_list = []
name_list = []
symbol_list = []
slug_list = []
price_list = []
mkt_cap_list = []
market_cap_dominance_list = []
platform_list = []

for i in range(1000):
  if i < 500:
    type_list.append('coin')
    platform_list.append(np.nan)
    name_list.append(coin_data['data'][i]['name'])
    symbol_list.append(coin_data['data'][i]['symbol'])
    slug_list.append(coin_data['data'][i]['slug'])
    price_list.append(coin_data['data'][i]['quote']['USD']['price'])
    mkt_cap_list.append(coin_data['data'][i]['quote']['USD']['market_cap'])
    market_cap_dominance_list.append(coin_data['data'][i]['quote']['USD']['market_cap_dominance'])
  else:
    type_list.append('token')
    try: 
      platform_list.append(token_data['data'][i-500]['platform']['name'])
    except: 
      platform_list.append(np.nan)
    name_list.append(token_data['data'][i-500]['name'])
    symbol_list.append(token_data['data'][i-500]['symbol'])
    slug_list.append(token_data['data'][i-500]['slug'])
    price_list.append(token_data['data'][i-500]['quote']['USD']['price'])
    mkt_cap_list.append(token_data['data'][i-500]['quote']['USD']['market_cap'])
    market_cap_dominance_list.append(token_data['data'][i-500]['quote']['USD']['market_cap_dominance'])
full_df['type'] = type_list
full_df['platform'] = platform_list
full_df['name'] = name_list
full_df['symbol'] = symbol_list
full_df['slug'] = slug_list
full_df['price'] = price_list
full_df['mkt_cap'] = mkt_cap_list
full_df['market_cap_dominance'] = market_cap_dominance_list


In [ ]:
full_df.head()

,type,platform,name,symbol,slug,price,mkt_cap,market_cap_dominance
0,coin,NaN,Bitcoin,BTC,bitcoin,48351.799188,9.137360e+11,40.9107
1,coin,NaN,Ethereum,ETH,ethereum,4025.286997,4.777614e+11,21.3857
2,coin,NaN,Binance Coin,BNB,binance-coin,565.297718,9.429231e+10,4.2226
3,coin,NaN,Solana,SOL,solana,169.926571,5.225465e+10,2.3400
4,coin,NaN,Cardano,ADA,cardano,1.239728,4.144291e+10,1.8551


# Decentralization Score

In [ ]:
# scrape data from coincheckup.com externally using Octopus
urls = ['https://coincheckup.com/coins/{}/analysis'.format(i) for i in full_df['slug']]

Decentralization score is gathered on Coincheckup.com through web scraping. However, since Coincheckup does the scoring manually, the number of coins/tokens with a score is limited. We need to drop the nan data.

In [ ]:
full_df = pd.read_csv('full_df_with_score.csv')
full_df = full_df[full_df['dec_score'].notna()]
full_df = full_df.reset_index().drop(columns='index')
full_df['dec_score'] = [full_df['dec_score'][i][15:-15] for i in range(len(full_df))] 
full_df

,type,platform,name,symbol,slug,price,mkt_cap,market_cap_dominance,dec_score
0,coin,NaN,Bitcoin,BTC,bitcoin,48351.799190,9.137360e+11,40.9107,Decentralized
1,coin,NaN,Ethereum,ETH,ethereum,4025.286997,4.777610e+11,21.3857,Semi-centralized
2,coin,NaN,Binance Coin,BNB,binance-coin,565.297718,9.429231e+10,4.2226,Centralized > Flat
3,coin,NaN,Solana,SOL,solana,169.926571,5.225465e+10,2.3400,Centralized > Hierarchical
4,coin,NaN,Cardano,ADA,cardano,1.239728,4.144291e+10,1.8551,Semi-centralized
...,...,...,...,...,...,...,...,...,...
377,token,NaN,SIX,SIX,six,0.170793,4.675914e+07,0.0021,Centralized > Hierarchical
378,token,Ethereum,SOLVE,SOLVE,solve,0.106309,4.606652e+07,0.0021,Centralized > Hierarchical
379,token,Ethereum,Ultiledger,ULT,ultiledger,0.015006,4.256267e+07,0.0000,Decentralized
380,token,Ethereum,ForTube,FOR,the-force-protocol,0.063619,4.068534e+07,0.0018,Decentralized


Ranking the categories from most decentralized to least decentralized: <br>'Decentralized', 'Semi-centralized', 'Centralized > Flat', 'Centralized > Hierarchical'

In [ ]:
# drop a few duplicated symbol
full_df = full_df[~full_df.duplicated(keep='last',subset=['symbol'])]

In [ ]:
print('There are {} data points of coin (layer1) and {} data points of token (layer2)'.format(len(full_df[full_df['type'] == 'coin']), len(full_df[full_df['type'] == 'token'])))

There are 261 data points of coin (layer1) and 117 data points of token (layer2)


In [110]:
# full_df.to_csv('full_df.csv')

# Platform Performance

In [7]:
!pip install sanpy
!pip install --upgrade sanpy

     |████████████████████████████████| 10.8 MB 14.1 MB/s 
  Attempting uninstall: pandas
    Found existing installation: pandas 1.1.5
    Uninstalling pandas-1.1.5:
      Successfully uninstalled pandas-1.1.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas~=1.1.0; python_version >= "3.0", but you have pandas 1.3.0 which is incompatible.


# Get other performance metrics from Santiment

In [4]:
import san
san.ApiConfig.api_key = 'wk2dgerkccxbuxea_6ldwzpn7kdk37lsn'

In [5]:
san_all = san.get("projects/all")

In [7]:
# drop duplicated tickers in san_all dataframe
common_symbols = full_df[full_df['symbol'].isin(san_all['ticker'])]['symbol'].values

tmp = san_all[san_all['ticker'].isin(common_symbols)]
san_dup = tmp[tmp.duplicated(keep=False,subset=['ticker'])].sort_values(by='ticker')
san_dup = san_dup[~san_dup.name.isin(full_df['name'])].drop(index=1267)

# cleaned san_df
san_df = pd.concat([tmp, san_dup, san_dup]).drop_duplicates(keep=False)

In [ ]:
metrics_df = pd.DataFrame()
cnt = 0
for i in san_df['slug']:
  try:
    marketcap_usd = san.get(
      "marketcap_usd/{}".format(i),
      from_date="2019-12-11",
      to_date="2021-12-11",
      interval="1d")
  except:
    pass

  try:
    marketcap_usd.columns = ['{}_marketcap_usd'.format(i)]
  except:
    pass
  
  try:
    amount_in_top_holders = san.get(
        "amount_in_top_holders/{}".format(i),
        from_date="2019-12-11",
        to_date="2021-12-11",
        interval="1d")
  except:
    pass
  try:
    amount_in_top_holders.columns = ['{}_amount_in_top_holders'.format(i)]
  except:
    pass
  try:
    community_messages_count_total = san.get(
        "community_messages_count_total/{}".format(i),
        from_date="2019-12-11",
        to_date="2021-12-11",
        interval="1d")
  except:
    pass

  try:
    community_messages_count_total.columns = ['{}_community_messages_count_total'.format(i)]
  except:
    pass
  try:
    social_volume_total = san.get(
        "social_volume_total/{}".format(i),
        from_date="2019-12-11",
        to_date="2021-12-11",
        interval="1d")
  except:
    pass

  try:
    social_volume_total.columns = ['{}_social_volume_total'.format(i)]
  except:
    pass
  try:
    sentiment_positive_total = san.get(
        "sentiment_positive_total/{}".format(i),
        from_date="2019-12-11",
        to_date="2021-12-11",
        interval="1d")
  except:
    pass
  try:
    sentiment_positive_total.columns = ['{}_sentiment_positive_total'.format(i)]
  except:
    pass
  
  try:
    sentiment_negative_total = san.get(
        "sentiment_negative_total/{}".format(i),
        from_date="2019-12-11",
        to_date="2021-12-11",
        interval="1d")
  except:
    pass
  try:
    sentiment_negative_total.columns = ['{}_sentiment_negative_total'.format(i)]
  except:
    pass

  try:
    dev_activity = san.get(
        "dev_activity/{}".format(i),
        from_date="2019-12-11",
        to_date="2021-12-11",
        interval="1d")
  except:
    pass
  try:
    dev_activity.columns = ['{}_dev_activity'.format(i)]
  except:
    pass

  try:
    github_activity = san.get(
        "github_activity/{}".format(i),
        from_date="2019-12-11",
        to_date="2021-12-11",
        interval="1d")
  except:
    pass
  try:
    github_activity.columns = ['{}_github_activity'.format(i)]
  except:
    pass
  
  try:
    dev_activity_contributors_count = san.get(
        "dev_activity_contributors_count/{}".format(i),
        from_date="2019-12-11",
        to_date="2021-12-11",
        interval="1d")
  except:
    pass
  try:
    dev_activity_contributors_count.columns = ['{}_dev_activity_contributors_count'.format(i)]
  except:
    pass
  
  slug_df = pd.concat([marketcap_usd,amount_in_top_holders,community_messages_count_total,
                       social_volume_total,sentiment_positive_total,sentiment_negative_total,
                       dev_activity,github_activity,dev_activity_contributors_count],axis=1)
  
  metrics_df = pd.concat([metrics_df,slug_df],axis=1)
  cnt+=1
  print(cnt)

In [9]:
# metrics_df.to_csv('metrics_df.csv')
metrics_df = pd.read_csv('metrics_df.csv')

In [13]:
features = ['marketcap_usd','amount_in_top_holders',
                       'social_volume_total','sentiment_positive_total','sentiment_negative_total',
                       'dev_activity','github_activity','dev_activity_contributors_count']
intersect = []
for i in features:
    cnt = 0
    coins = []
    for j in metrics_df.columns:
        if j[-len(i):] == i:
            cnt+=1
            coins.append(j[:-len(i)-1])
    if i == features[0]:
        intersect = coins
    else:
        intersect = [value for value in intersect if value in coins]
    print("{}:{}".format(i,cnt))

print('There are {} cryptos with all eight features'.format(len(intersect)))

marketcap_usd:298
amount_in_top_holders:292
social_volume_total:298
sentiment_positive_total:298
sentiment_negative_total:298
dev_activity:264
github_activity:264
dev_activity_contributors_count:264
There are 253 cryptos with all eight features


In [82]:
# create a multi-index dataframe, index1: slug, index2:date
date = [pd.to_datetime(metrics_df['datetime'])[i].strftime('%Y-%m-%d') for i in range(len(metrics_df))]

slug = intersect
iterables = [slug,date]

idx = pd.MultiIndex.from_product(iterables, names=["slug", "date"])

col_dict = dict.fromkeys(features,)
for i in features:
  tmp = []
  for crypto in intersect:
    tmp.extend(metrics_df[crypto+'_'+i].values)
  col_dict[i] = tmp

In [83]:
performance_df = pd.DataFrame(col_dict, index=idx)
performance_df

marketcap_usd  ...  dev_activity_contributors_count
slug     date                       ...                                 
enecuum  2019-12-11       939828.0  ...                              0.0
         2019-12-12       978550.0  ...                              0.0
         2019-12-13      1009804.0  ...                              0.0
         2019-12-14       931420.0  ...                              0.0
         2019-12-15       914745.0  ...                              0.0
...                            ...  ...                              ...
sapphire 2021-12-07    242854453.0  ...                              0.0
         2021-12-08    241091714.0  ...                              0.0
         2021-12-09    227697891.0  ...                              0.0
         2021-12-10    226242364.0  ...                              0.0
         2021-12-11    235822786.0  ...                              NaN

[185196 rows x 8 columns]

In [104]:
merged_slug_241 = full_df[full_df['slug'].isin(slug)].slug.values
diff = [i for i in slug if i not in merged_slug_241]
diff_df = san_all[san_all['slug'].isin(diff)]

In [108]:
final_df = pd.concat([full_df[full_df['slug'].isin(slug)],full_df[full_df['symbol'].isin(diff_df.ticker)]])
final_df

,type,platform,name,symbol,slug,price,mkt_cap,market_cap_dominance,dec_score
0,coin,NaN,Bitcoin,BTC,bitcoin,48351.799190,9.137360e+11,40.9107,Decentralized
1,coin,NaN,Ethereum,ETH,ethereum,4025.286997,4.777610e+11,21.3857,Semi-centralized
2,coin,NaN,Binance Coin,BNB,binance-coin,565.297718,9.429231e+10,4.2226,Centralized > Flat
3,coin,NaN,Solana,SOL,solana,169.926571,5.225465e+10,2.3400,Centralized > Hierarchical
4,coin,NaN,Cardano,ADA,cardano,1.239728,4.144291e+10,1.8551,Semi-centralized
...,...,...,...,...,...,...,...,...,...
168,coin,NaN,INT,INT,int-chain,0.011176,5.438173e+06,0.0002,Centralized > Flat
225,coin,NaN,Phoenixcoin,PXC,phoenixcoin,0.014503,1.237056e+06,0.0000,Decentralized
351,token,Ethereum,Streamr,DATA,streamr,0.118340,9.078092e+07,0.0041,Centralized > Hierarchical
356,token,Ethereum,DAD,DAD,dad,0.175119,8.338439e+07,0.0037,Centralized > Hierarchical


In [133]:
# final_df.drop(columns='price').to_csv('final_df.csv')
# performance_df.to_csv('performance_df.csv')

In [137]:
performance_df.shape

(185196, 8)